In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({
    'age': [17, 64, 18, 20, 38, 49, 55, 25, 29, 31],
    'income': [25, 80, 22, 36, 37, 59, 74, 70, 33, 102],
    'loan': [1, 0, 1, 0, 1, 0, 0, 1, 1, 0]
})
x = data[['age', 'income']]
y = data['loan']

1. Сортируем признак по возрастанию и удаляем дубликаты
2. Находим скользящее среднее и удаляем пустые значения (первое всегда пустое)
3. Находим индексы y, которые соответствуют скользящим средним
4. Находим разницу по y с целью того, чтобы понять, где класс меняется
5. Выбираем те индексы и соответствующие им средние, у которых класс y изменился

In [3]:
def find_candidates_for_thresholds(x, y):
    x = x.sort_values().drop_duplicates()
    x_rolling_mean = x.rolling(2).mean().dropna()
    y = y[x_rolling_mean.index]
    y_diff = y.diff()
    candidades = x_rolling_mean[y_diff!=0]
    return candidades.values

Среднеквадратичная ошибка:

In [4]:
def squared_error(y):
    y_mean = y.mean()
    mse = ((y-y_mean)**2).mean()
    return mse


Обеспечивается разбиение датафрейма по указанным параметрам:

In [5]:
def split(X, y, split_params):
    j, t = split_params
    predicat = X.iloc[:, j]<=t
    X_left, y_left = X[predicat], y[predicat]
    X_right, y_right = X[~predicat], y[~predicat]
    return X_left, y_left, X_right, y_right

Вычисляется взвешенная неоднородность после разбиения:

In [6]:
def calculate_weighted_impurity(X, y, split_params, criterion):
    X_left, y_left, X_right, y_right = split(X, y, split_params)
    N, N_left, N_right = y.size, y_left.size, y_right.size
    score = (N_left/N)*criterion(y_left) + (N_right/N)*criterion(y_right)
    return score

Ищется оптимальное разбиение:

In [7]:
def best_split(X, y, criterion):
    M = X.shape[1]
    min_weighted_impurity = np.inf
    optimal_split_params = None

    for j in range(M):
        candidades = find_candidates_for_thresholds(X.iloc[:,j], y)
        for t in candidades:
            eighted_impurity = calculate_weighted_impurity(X, y, (j, t), squared_error)
            if eighted_impurity < min_weighted_impurity:
                min_weighted_impurity = eighted_impurity
                optimal_split_params = (j ,t)

    return optimal_split_params

In [8]:
data = pd.DataFrame({
    'x1': list(range(1, 16)),
    'y': [1, 1.2, 1.4, 1.1, 1, 5.5, 6.1, 6.7, 6.4, 6, 6, 3, 3.2, 3.1, 3]
})
X = data[['x1']]
y = data['y']

In [9]:
optimal_split_params = best_split(X, y, criterion=squared_error)
print('Optimal j: {}'.format(optimal_split_params[0]))
print('Optimal t: {}'.format(optimal_split_params[1]))

Optimal j: 0
Optimal t: 5.5


Энтропия Шеннона. Минус (сумма по k=1 до K по Pk*log2(Pk)):

In [10]:
def entropy(y):
    p = y.value_counts(normalize = True)
    entropy = np.sum(p*np.log2(p))
    return entropy

In [11]:
class Node:
    def __init__(self, left=None, 
                 right=None, value=None, 
                 split_params=None, impurity=None,
                 samples=None, is_leaf=False):
        self.left = left
        self.right = right
        self.split_params = split_params
        self.value = value
        self.impurity = impurity
        self.samples = samples
        self.is_leaf = is_leaf